In [175]:
import pandas as pd
import untangle # xml
import requests # json
import re # regular expressions

In [176]:
# Configuration
our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]

In [186]:
# Pull the XML and traverse to the blobs
xmlblobs = untangle.parse(
    'https://publicdashacc.blob.core.windows.net/publicdata?restype=container&comp=list'
).EnumerationResults.Blobs.Blob

In [202]:
# Extract the cdata of the names, filter to data_000….json, convert to list, sort descending
jsons = sorted(
    list(
        filter(
            lambda b: re.search('^data_[0-9]+.json$',b),
            map(
                lambda b: b.Name.cdata,
                xmlblobs
            )
        )
    ),reverse=False
)

In [203]:
# Grab the last JSON in the sorted list, append it to the URL
latest_json_url = 'https://c19pub.azureedge.net/' + jsons[-1]

In [206]:
# Pull the JSON and convert to a Python object
json = requests.get(latest_json_url).json()

In [198]:
# Filter to MNA's UTLAs
new_data = []
for (code, data) in json['utlas'].items():
    if code in our_utla_codes:
        for day in data['dailyConfirmedCases']:
            new_data.append([
                code,
                data['name']['value'],
                day['date'],
                day['value']
            ])

In [199]:
df = pd.DataFrame(data=new_data, columns=['code','name','date','new cases'])